# Дектораторы

В этом домашнем задании мы напишем собственные дектораторы, которые будут менять системные объекты. Но для начала мы с вами познакомимся с функцией `write`.

In [10]:
import sys

sys.stdout.write('Hello, my friend!')

Hello, my friend!

17

Это метод объектов file-like классов, то есть классов, которые реализуют семантику "Меня можно создать, из меня можно прочитать и в меня можно записать".

Самый главный пример такого объекта -- объект `file`, являющийся результатом вызова фукнции `open()`. Для простоты и универсальности взаимодействия, стандартный ввод и стандартный вывод тоже являются файлами, из которых можно читать и в которые можно писать. 

In [11]:
output = open("./some_test_data.txt", 'w')

In [12]:
output.write('123')

3

In [13]:
output.close()

Как вы могли заметить, функция возвращает число записанных байт. Это важная часть контракта, которую нужно поддержать, если вы хотите как-то подменять эту функцию.

## Задача 1

Для начала, давайте подменим метод `write` у объекта `sys.stdin` на такую функцию, которая перед каждым вызовом оригинальной функции записи данных в `stdout` допечатывает к тексту текущую метку времени.

In [14]:
from datetime import datetime
original_write = sys.stdout.write

def my_write(string_text):
    if string_text != '\n':
        current_datetime = datetime.now()
        current_datetime_string = current_datetime.strftime("[%Y-%m-%d %H:%M:%S]: ")
        return original_write(f'{current_datetime_string}{string_text}')
    else:
        return original_write(f'{string_text}')

sys.stdout.write = my_write

In [15]:
print('1, 2, 3')

[2021-12-10 21:02:02]: 1, 2, 3


In [16]:
sys.stdout.write = original_write

Вывод должен был бы быть примерно таким:

```
[2021-12-05 12:00:00]: 1, 2, 3
```

## Задача 2

Упакуйте только что написанный код в декторатор. Весь вывод фукнции должен быть помечен временными метками так, как видно выше.

In [17]:
def timed_output(function):
    original_write = sys.stdout.write
    def decorated(some_str):
        def my_write(some_str):
            if some_str != '\n':
                current_datetime = datetime.now()
                current_datetime_string = current_datetime.strftime("[%Y-%m-%d %H:%M:%S]: ")
                return original_write(f'{current_datetime_string}{some_str}')
            else:
                sys.stdout.write = original_write
                return original_write(some_str)
        sys.stdout.write = my_write
        return function(some_str)
    return decorated

In [18]:
@timed_output
def print_greeting(name):
    print(f'Hello, {name}!')

In [19]:
print_greeting("Nikita")
print_greeting("Sveta")
print("Nataha")

[2021-12-10 21:02:06]: Hello, Nikita!
[2021-12-10 21:02:06]: Hello, Sveta!
Nataha


Вывод должен быть похож на следующий:

```
[2021-12-05 12:00:00]: Hello, Nikita!
```

## Задача 3

Напишите декторатор, который будет перенаправлять вывод фукнции в файл. 

Подсказка: вы можете заменить объект sys.stdout каким-нибудь другим объектом.

In [20]:
def redirect_output(filepath):
    original_out = sys.stdout
    def decorator(function):
        def decorated():
            output = open(filepath, 'w')
            sys.stdout = output
            fun = function()
            sys.stdout = original_out
            output.close()
            return fun
        return decorated
    return decorator

@redirect_output('./function_output.txt')
def calculate():
    for power in range(1, 5):
        for num in range(1, 20):
            print(num ** power, end=' ')
        print()

calculate()

In [21]:
@redirect_output('./function_output.txt')
def calculate():
    for power in range(1, 5):
        for num in range(1, 20):
            print(num ** power, end=' ')
        print()

In [22]:
calculate()

In [23]:
%cat function_output.txt

UsageError: Line magic function `%cat` not found.
